<a href="https://colab.research.google.com/github/LucasOsco/AI-RemoteSensing/blob/main/sam_text_v01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install the necessary libraries
!pip install rasterio
!pip install torch torchvision
!pip install segment-anything
!pip install huggingface_hub
!pip install -U git+https://github.com/IDEA-Research/GroundingDINO.git

In [ ]:
# Import required libraries
import os
import numpy as np
import torch
import rasterio
import matplotlib.pyplot as plt
import groundingdino.datasets.transforms as T
from PIL import Image
from rasterio.plot import show
from matplotlib.patches import Rectangle
from groundingdino.models import build_model
from groundingdino.util import box_ops
from groundingdino.util.inference import predict
from groundingdino.util.slconfig import SLConfig
from groundingdino.util.utils import clean_state_dict
from huggingface_hub import hf_hub_download
from segment_anything import sam_model_registry
from segment_anything import SamPredictor

In [ ]:
# Define constants
SAM_MODELS = {
    "vit_h": "https://dl.fbaipublicfiles.com/segment_anything/sam_vit_h_4b8939.pth",
    "vit_l": "https://dl.fbaipublicfiles.com/segment_anything/sam_vit_l_0b3195.pth",
    "vit_b": "https://dl.fbaipublicfiles.com/segment_anything/sam_vit_b_01ec64.pth"
}

CACHE_PATH = os.environ.get("TORCH_HOME", os.path.expanduser("~/.cache/torch/hub/checkpoints"))

In [ ]:
# Define helper functions
def load_model_hf(repo_id, filename, ckpt_config_filename, device='cpu'):
    cache_config_file = hf_hub_download(repo_id=repo_id, filename=ckpt_config_filename)
    args = SLConfig.fromfile(cache_config_file)
    model = build_model(args)
    model.to(device)
    cache_file = hf_hub_download(repo_id=repo_id, filename=filename)
    checkpoint = torch.load(cache_file, map_location='cpu')
    model.load_state_dict(clean_state_dict(checkpoint['model']), strict=False)
    model.eval()
    return model

def transform_image(image) -> torch.Tensor:
    transform = T.Compose([
        T.RandomResize([800], max_size=1333),
        T.ToTensor(),
        T.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
    ])
    image_transformed, _ = transform(image, None)
    return image_transformed

In [ ]:
# Class definition for LangSAM
class LangSAM():
    def __init__(self, sam_type="vit_h"):
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.build_groundingdino()
        self.build_sam(sam_type)

    def build_sam(self, sam_type):
        checkpoint_url = SAM_MODELS[sam_type]
        sam = sam_model_registry[sam_type]()
        state_dict = torch.hub.load_state_dict_from_url(checkpoint_url)
        sam.load_state_dict(state_dict, strict=True)
        sam.to(device=self.device)
        self.sam = SamPredictor(sam)

    def build_groundingdino(self):
        ckpt_repo_id = "ShilongLiu/GroundingDINO"
        ckpt_filenmae = "groundingdino_swinb_cogcoor.pth"
        ckpt_config_filename = "GroundingDINO_SwinB.cfg.py"
        self.groundingdino = load_model_hf(ckpt_repo_id, ckpt_filenmae, ckpt_config_filename, self.device)

    def predict_dino(self, image_pil, text_prompt, box_threshold, text_threshold):
        image_trans = transform_image(image_pil)
        boxes, logits, phrases = predict(model=self.groundingdino,
                                         image=image_trans,
                                         caption=text_prompt,
                                         box_threshold=box_threshold,
                                         text_threshold=text_threshold,
                                         device=self.device)
        W, H = image_pil.size
        boxes = box_ops.box_cxcywh_to_xyxy(boxes) * torch.Tensor([W, H, W, H])

        return boxes, logits, phrases

    def predict_sam(self, image_pil, boxes):
        image_array = np.asarray(image_pil)
        self.sam.set_image(image_array)
        transformed_boxes = self.sam.transform.apply_boxes_torch(boxes, image_array.shape[:2])
        masks, _, _ = self.sam.predict_torch(
            point_coords=None,
            point_labels=None,
            boxes=transformed_boxes.to(self.sam.device),
            multimask_output=False,
        )
        return masks.cpu()

    def predict(self, image_pil, text_prompt, box_threshold, text_threshold):
        boxes, logits, phrases = self.predict_dino(image_pil, text_prompt, box_threshold, text_threshold)
        masks = torch.tensor([])
        if len(boxes) > 0:
            masks = self.predict_sam(image_pil, boxes)
            masks = masks.squeeze(1)
        return masks, boxes, phrases, logits

In [ ]:
# Load the georeferenced image
with rasterio.open("Image.tif") as src:
    image_np = src.read().transpose((1, 2, 0))  # Convert rasterio image to numpy array
    transform = src.transform  # Save georeferencing information
    crs = src.crs  # Save the Coordinate Reference System

# Provide the text-prompt
text_prompt = "tree"

In [ ]:
# Initialize the model
model = LangSAM()

In [ ]:
image_pil = Image.fromarray(image_np[:, :, :3]) # Convert numpy array to PIL image, excluding the alpha channel
image_np_copy = image_np.copy()  # Create a copy for modifications

masks, boxes, phrases, logits = model.predict(image_pil, text_prompt, box_threshold=0.3, text_threshold=0.3) # Customize the box and text threshold values to your data

if boxes.nelement() == 0:  # No "object" instances found
    print('No objects found in the image.')
else:
    # Create an empty image to store the mask overlays
    mask_overlay = np.zeros_like(image_np[..., 0], dtype=np.int64)  # Adjusted for single channel

    for i in range(len(boxes)):
        box = boxes[i].cpu().numpy()  # Convert the tensor to a numpy array
        mask = masks[i].cpu().numpy()  # Convert the tensor to a numpy array

        # Add the mask to the mask_overlay image
        mask_overlay += ((mask > 0) * (i + 1))  # Assign a unique value for each mask

    # Normalize mask_overlay to be in [0, 255]
    mask_overlay = (mask_overlay > 0) * 255  # Binary mask in [0, 255]

    # Display the original image with all mask overlays and bounding boxes
    plt.figure(figsize=(10, 10))
    plt.imshow(image_pil)

    for i in range(len(boxes)):
        box = boxes[i].cpu().numpy()  # Convert the tensor to a numpy array
        # Draw bounding box
        rect = Rectangle((box[0], box[1]), box[2] - box[0], box[3] - box[1], linewidth=1, edgecolor='r', facecolor='none')
        plt.gca().add_patch(rect)

    plt.imshow(mask_overlay, cmap='viridis', alpha=0.5)  # Overlay the mask with some transparency
    plt.title(f"Segmented")
    plt.show()

In [ ]:
# Save the image as a GeoTIFF
mask_overlay = ((mask_overlay > 0) * 255).astype(rasterio.uint8)  # Binary mask in [0, 255]

# Save the mask_overlay as a new georeferenced raster
with rasterio.open(
    'mask.tif',
    'w',
    driver='GTiff',
    height=mask_overlay.shape[0],
    width=mask_overlay.shape[1],
    count=1,
    dtype=mask_overlay.dtype,
    crs=crs,
    transform=transform,
) as dst:
    dst.write(mask_overlay, 1)